# Quels concepts

- 

- 

- 

- 

# Analyse de la profondeur de champ

In [16]:
import pandas as pd
import numpy as np
from  pathlib import Path
import os

In [17]:
r = 1/1.8 # en pouces
r = r*2.54 # en cm
r = r*10**-2 #en m
df = pd.DataFrame(columns=["image", "pdc_observe", "f", "k", "g"])

In [18]:
pdc_path = "TP1_Donnee/Pdc/"
imgs = [str(img) for img in Path(pdc_path).glob("*.JPG")]
df["image"] = imgs
df

,image,pdc_observe,f,k,g
0,TP1_Donnee/Pdc/PICT3149.JPG,NaN,NaN,NaN,NaN
1,TP1_Donnee/Pdc/PICT3153.JPG,NaN,NaN,NaN,NaN
2,TP1_Donnee/Pdc/PICT3152.JPG,NaN,NaN,NaN,NaN
3,TP1_Donnee/Pdc/PICT3150.JPG,NaN,NaN,NaN,NaN
4,TP1_Donnee/Pdc/PICT3151.JPG,NaN,NaN,NaN,NaN
5,TP1_Donnee/Pdc/PICT3154.JPG,NaN,NaN,NaN,NaN


In [19]:
from PIL import Image
from PIL.ExifTags import TAGS

def get_exif_data(image_path):
    # Open the image file
    img = Image.open(image_path)
    
    # Extract EXIF data
    exif_data = img._getexif()
    
    # Convert EXIF tags to human-readable form
    exif_info = {}
    if exif_data:
        for tag, value in exif_data.items():
            tag_name = TAGS.get(tag, tag)
            exif_info[tag_name] = value
            
    return exif_info

def extract_metadata(image_path):
    # Get EXIF data
    exif_info = get_exif_data(image_path)
    
    # Access specific metadata fields
    focal_length = exif_info.get('FocalLength', 'N/A')
    f_number = exif_info.get('FNumber', 'N/A')
    
    return focal_length, f_number

meta_data = get_exif_data(pdc_path + "PICT3153.JPG")

meta_data_key_map = {
    "f" : "FocalLength",
    "g" : "FocalLengthIn35mmFilm",
    "k" : "FNumber"
}

In [20]:
def extract_variable(row : pd.Series):
    img_name  = row.iloc[0]
    meta_data = get_exif_data(img_name)
    for key in meta_data_key_map:
        row.loc[key] = float(meta_data[meta_data_key_map[key]])
    return row


#extract_variable(row)
df = df.apply(extract_variable, axis=1)
df[["f","g", "k"]] *= 10**-3
df

,image,pdc_observe,f,k,g
0,TP1_Donnee/Pdc/PICT3149.JPG,NaN,0.007203,3.2,0.028
1,TP1_Donnee/Pdc/PICT3153.JPG,NaN,0.007203,5.6,0.028
2,TP1_Donnee/Pdc/PICT3152.JPG,NaN,0.007203,11.0,0.028
3,TP1_Donnee/Pdc/PICT3150.JPG,NaN,0.007203,2.8,0.028
4,TP1_Donnee/Pdc/PICT3151.JPG,NaN,0.007203,11.0,0.028
5,TP1_Donnee/Pdc/PICT3154.JPG,NaN,0.007203,2.8,0.028


In [21]:
def compute_pdc(row : pd.Series):
    f, k, g = row["f"], row["k"], row["g"] 

    num = 4*(f**2) * r*k*g*(g-f)
    dnm = f**4 - 4*(r**2)*(k**2)*(g-f)**2
    row["pdc_calcule"] = num/dnm
    return row

df = df.apply(compute_pdc, axis=1)
df

,image,pdc_observe,f,k,g,pdc_calcule
0,TP1_Donnee/Pdc/PICT3149.JPG,NaN,0.007203,3.2,0.028,-0.001548
1,TP1_Donnee/Pdc/PICT3153.JPG,NaN,0.007203,5.6,0.028,-0.000884
2,TP1_Donnee/Pdc/PICT3152.JPG,NaN,0.007203,11.0,0.028,-0.000450
3,TP1_Donnee/Pdc/PICT3150.JPG,NaN,0.007203,2.8,0.028,-0.001770
4,TP1_Donnee/Pdc/PICT3151.JPG,NaN,0.007203,11.0,0.028,-0.000450
5,TP1_Donnee/Pdc/PICT3154.JPG,NaN,0.007203,2.8,0.028,-0.001770


In [22]:
dft = df[["f", "k", "g"]]
num = 4*dft["f"]**2 * r*dft["k"]*dft["g"]*(dft["g"]-dft["f"])
dnm = dft["f"]**4 - 4*r**2*dft["k"]**2*(dft["g"]-dft["f"])**2
computed_pdc = num/dnm
df["pdc_calcule"] = computed_pdc
df

,image,pdc_observe,f,k,g,pdc_calcule
0,TP1_Donnee/Pdc/PICT3149.JPG,NaN,0.007203,3.2,0.028,-0.001548
1,TP1_Donnee/Pdc/PICT3153.JPG,NaN,0.007203,5.6,0.028,-0.000884
2,TP1_Donnee/Pdc/PICT3152.JPG,NaN,0.007203,11.0,0.028,-0.000450
3,TP1_Donnee/Pdc/PICT3150.JPG,NaN,0.007203,2.8,0.028,-0.001770
4,TP1_Donnee/Pdc/PICT3151.JPG,NaN,0.007203,11.0,0.028,-0.000450
5,TP1_Donnee/Pdc/PICT3154.JPG,NaN,0.007203,2.8,0.028,-0.001770


In [23]:
pdc_observe = [3.9, 15, 16, 5, 14.4, 8] # mettre ici mes observation
df["pdc_observe"] = pdc_observe
df["erreur"] = np.abs(df["pdc_observe"] - df["pdc_calcule"])
df

,image,pdc_observe,f,k,g,pdc_calcule,erreur
0,TP1_Donnee/Pdc/PICT3149.JPG,3.9,0.007203,3.2,0.028,-0.001548,3.901548
1,TP1_Donnee/Pdc/PICT3153.JPG,15.0,0.007203,5.6,0.028,-0.000884,15.000884
2,TP1_Donnee/Pdc/PICT3152.JPG,16.0,0.007203,11.0,0.028,-0.000450,16.000450
3,TP1_Donnee/Pdc/PICT3150.JPG,5.0,0.007203,2.8,0.028,-0.001770,5.001770
4,TP1_Donnee/Pdc/PICT3151.JPG,14.4,0.007203,11.0,0.028,-0.000450,14.400450
5,TP1_Donnee/Pdc/PICT3154.JPG,8.0,0.007203,2.8,0.028,-0.001770,8.001770


# Analyse de la resolution

In [24]:
fto_path = ""
ftm_path = "TP1_Donnee/FTM"

imgs = [ str(img) for img in Path(ftm_path).glob("*.JPG")]

ftm_df = pd.DataFrame(columns=["image"])
ftm_df["image"] = imgs

ftm_df

,image
0,TP1_Donnee/FTM/PICT3148.JPG
1,TP1_Donnee/FTM/PICT3147.JPG
2,TP1_Donnee/FTM/PICT3146.JPG
3,TP1_Donnee/FTM/PICT3144.JPG
4,TP1_Donnee/FTM/PICT3145.JPG
5,TP1_Donnee/FTM/PICT3141.JPG
6,TP1_Donnee/FTM/PICT3140.JPG
7,TP1_Donnee/FTM/PICT3142.JPG
8,TP1_Donnee/FTM/PICT3143.JPG


In [25]:
meta_data = get_exif_data((ftm_df["image"]).iloc[0])




In [26]:
def extract_resoltion_data(row : pd.Series):
    img_path  = row.iloc[0]
    meta_data = get_exif_data(img_path)

    properties_to_extract =  [ key for key in meta_data.keys() if "Resol" in key]
    properties_to_extract += [ key for key in meta_data.keys() if ("Height" in key or "Width" in key)]
    
    for key in properties_to_extract:
        row.loc[key] = float(meta_data[key])
    return row

ftm_df = ftm_df.apply(extract_resoltion_data, axis=1)
ftm_df


,image,ResolutionUnit,XResolution,YResolution,ExifImageWidth,ExifImageHeight
0,TP1_Donnee/FTM/PICT3148.JPG,2.0,72.0,72.0,3264.0,2176.0
1,TP1_Donnee/FTM/PICT3147.JPG,2.0,72.0,72.0,3264.0,2176.0
2,TP1_Donnee/FTM/PICT3146.JPG,2.0,72.0,72.0,3264.0,2176.0
3,TP1_Donnee/FTM/PICT3144.JPG,2.0,72.0,72.0,3264.0,2176.0
4,TP1_Donnee/FTM/PICT3145.JPG,2.0,72.0,72.0,3264.0,2176.0
5,TP1_Donnee/FTM/PICT3141.JPG,2.0,72.0,72.0,3264.0,2176.0
6,TP1_Donnee/FTM/PICT3140.JPG,2.0,72.0,72.0,3264.0,2176.0
7,TP1_Donnee/FTM/PICT3142.JPG,2.0,72.0,72.0,3264.0,2176.0
8,TP1_Donnee/FTM/PICT3143.JPG,2.0,72.0,72.0,3264.0,2176.0
